Let's try to employ some tools for dimensionality reduction to our data a and see what comes out. In special, it is often  interesting to see what PCA tells us about the problem. NMF also frequently enconters useful "parts" to decompose our image, and always produces interesting results in datasets like MNIST or face analysis. We are specially interested here to see how these methods deal with one of the most basic facts about this dataset: sometimes we have one, and sometimes two dice.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline   
plt.rcParams['image.cmap'] = 'gray'

def read_vectors(*filenames):
    data = np.vstack(
        tuple(np.fromfile(filename, dtype=np.uint8).reshape(-1,401)
                      for filename in filenames))
    return data[:,1:], data[:,0]

X_train, y_train = read_vectors(*[
    "../input/snake-eyes/snakeeyes_{:02d}.dat".format(nn) for nn in range(2)])
X_test, y_test = read_vectors("../input/snake-eyes/snakeeyes_test.dat")

We are basing this on https://www.kaggle.com/chientien/mnist-pca-nmf-lda-t-sne, who performed these analyses on MNIST. Thanks!

# PCA

Let's first look at the power of the first few principal components to "explain" the data.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(random_state=0, whiten=True)
pca.fit(X_train);

In [ ]:
exp_var_cum = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(exp_var_cum.size), exp_var_cum)
plt.grid()

In [ ]:
plt.plot(range(exp_var_cum.size), exp_var_cum, '-+')
plt.grid()
plt.xlim(15,105)
plt.ylim(0.6,0.95);

The result does not seem very different from MNIST. We could argue none of these image datasets can be _really_ explained by PCA, so neither of them are 100% explained by the e.g. first 10 components. This actually tell us something about the problem being very much non-convex, because our data does indeed fit very constrained manifolds, since most of the variation comes from the 3-DOF transforms over the dice faces. PCA does not seem to be able to grasp this fact, though, and this is because these "thin" manifolds are directed everywhere, and not just at a single direction that PCA would be able to find.

Anyway, I wonder what these principal components look like...

In [ ]:
plt.figure(figsize=(20,10))
for k in range(40):
    plt.subplot(4,10,k+1)
    plt.imshow(pca.components_[k].reshape(20,20))
    plt.axis('off')

Cool! Seems like we got ourselves some family of orthogonal basis functions here, something kind of like a Fourier transform or spheric Fourier transfom, who knows. Our very own Chladni patterns behind the good vibrations in this dataset. One may even wonder if instead of all the work and energy to calculate this from the data, we might not have just figured out these functions from formulas we found in a book that we read at school.

It is interesting to notice that although it was hard to see any difference to MNIST in the curves above, if we look at the actual principal components, on MNIST they still look a bit like combinations of the digits somehow. The figure below shows how the first MNIST principal components look like.

![MNIST principal components](https://i.imgur.com/9BGBKlj.png?1)

Using your imagination you can easily see the contous of a "0" or a "9" in there, while in our case we get something that looks more like some generic basis. Maybe this is the first clue we have that MNIST and Snake Eyes, actually have some deep differences about them. 

Let's cut the PCA to the lucky 90 components (that reach 90% "explainability"), and try some NN. I mean nearest neighbors, not neural networks.

In [ ]:
pca = PCA(n_components=90, random_state=0, whiten=True)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

But first I wonder: how do the PCA reconstructions look like?

In [ ]:
X_reconstructed_pca = pca.inverse_transform(X_test_pca)

plt.figure(figsize=(20,10))
for k in range(20):
    plt.subplot(4, 10, k*2 + 1)
    plt.imshow(X_test[k].reshape(20,20))
    plt.axis('off')
    plt.subplot(4, 10, k*2 + 2)
    plt.imshow(X_reconstructed_pca[k].reshape(20,20))
    plt.axis('off')

It's kinf of ugly, perhaps? Although the shape of the dice is there, and we can read pretty much all of the faces. We seem to have what we want: being able to tell what is what with less variables, only 22.5% of the original feature vector size. I tried 40 components before, and the faces look blurry. You can try out to play with this parameter later, here we are mostly concerned with making cool graphics, and 90 looks cooler.

But anyway, my opinion of how this data looks doesn't really matter, what is the opinion of NN?

In [ ]:
KNN_PCA_TRAIN_SIZE = 200000
KNN_PCA_TEST_SIZE = 200

from sklearn.neighbors import KNeighborsClassifier
temp = []
for i in [1, 5]:
    knn_pca = KNeighborsClassifier(n_neighbors=i, n_jobs=8)
    knn_pca.fit(X_train_pca[:KNN_PCA_TRAIN_SIZE], y_train[:KNN_PCA_TRAIN_SIZE])
    train_score_pca = knn_pca.score(X_train_pca[:KNN_PCA_TEST_SIZE], y_train[:KNN_PCA_TEST_SIZE])
    test_score_pca = knn_pca.score(X_test_pca[:KNN_PCA_TEST_SIZE], y_test[:KNN_PCA_TEST_SIZE])
    li = [i,train_score_pca,test_score_pca]
    temp.append(li)
temp

We limited the amount of data used in this experiment because NN inference takes a looong time to run. It's actually pretty much the only reason you might like to use it with the PCA transformed vectors instead of the original ones: to save on computation time. There is actually absolutely no guarantees that using PCA for this is a good idea, it is just a very desperate move. Every time you see someone applying PCA straight away for classification you should be sympathetic to this person, because they are most certainly desperate to run this classifier model on this data, but have a very constrained computational budget, and then see themselves forced to make this simplification that in many cases is actually bad for classification. Heck, we don't even know if Euclidean distance [is the good metric](http://yann.lecun.com/exdb/publis/pdf/simard-00.pdf) to employ here.

Sometimes the information you are throwing away when you calculate the PCA is precisely the information that might help you classify the data, and not merely compress i.e. minimize the reconstruction error. This is what LDA is about, finding out the direction that really matters for classification, and not for reconstruction. These two can be very different problems, representation and classification. In fact, one might wonder if finding out this encoding that would let you do a good classification is not the ultimate goal of machine learning.

I see now the NN test is over, and even though we did not use all our data, we can see an interesting result, we got 50% accuracy. Much better than the LDA [we tried before](https://www.kaggle.com/nicw102168/plotting-the-data-and-a-simple-demo). This shows the power of nearest neighbor, and how being more flexible than LDA is important for this problem. This is much more evident here than on MNIST, where LDA does not fare so bad.  But anyway, this NN seems quite slow. There are many things you can do to speed it up, though, but we will not look at this right now, let's stick to more basic analyses.

## NMF

If PCA is not such a terrific idea, perhaps we can try something else, something more non-linear, and something that has some good reasons to give good results in image analysis. Non-negative matrix factorization is just one such thing, and it's exciting because [according to some researchers](https://arxiv.org/abs/1509.05009) it even has some connections to Deep Learning, and Deep is exciting!

Let's train this NMF.

In [ ]:
NMF_TRAIN_SIZE = 100000

from sklearn.decomposition import NMF
nmf = NMF(n_components=90, random_state=0)
nmf.fit(X_train[:NMF_TRAIN_SIZE])

NMF training does take a lot of time too, I must say, but is it good for anything? Fair warning: we cut back again on the number of samples, so this is all just to have a taste of how these things work. Hard to say what would be the performance (as in speed and accuracy) on the full dataset and throwing all the GPUs we have at the problem.

Let's check again how the components look like. NMF is cool because it tends to find "parts" of things. That is the power of non-negativity. If you want to decompose your input into more "local" things than what we got with PCA, non-negativity is a very positive strategy!

And our dataset clearly has "parts": we are simulating these dice that are actual two physical parts that can move and rotate independently, and maybe NMF can find this. Our basis should hypotetically look just like what you would need to represent a single dice anywhere, and then we should be able to represent the two dice with that, because it is just a linear mixture.

In [ ]:
plt.figure(figsize=(20,10))
for k in range(40):
    plt.subplot(4, 10, k + 1)
    plt.imshow(nmf.components_[k].reshape(20,20))
    plt.axis('off')

Cool right? It does do something. If you use fewer components it clearly finds some large blobs the size of the dice. I suggest you experiment with it. With more components though, we find these tiny blobs moving around, and the rotated dice must be reconstructed from them somehow... It does look like parts of single dice, as we hypothesized earlier, nothing wide, the size of two dice or of the whole image.

These blobs are so tiny, tough, that it might be fair to say they are just working pretty much like chubby pixels. So maybe NMF is not doing much more than finding a representation of less resolution here. But it is doing it in a much more structured way than PCA.

Let's now see how the reconstructiion looks like.

In [ ]:
X_train_nmf = nmf.transform(X_train)
X_test_nmf = nmf.transform(X_test)
X_reconstructed = nmf.inverse_transform(X_test_nmf)

In [ ]:
plt.figure(figsize=(20,10))
for k in range(20):
    plt.subplot(4, 10, k*2 + 1)
    plt.imshow(X_test[k].reshape(20,20))
    plt.axis('off')
    plt.subplot(4, 10, k*2 + 2)
    plt.imshow(X_reconstructed[k].reshape(20,20))
    plt.axis('off')

So now we have a black background, in contrast to what PCA delivers, and the dice faces are kind of blurry in a different way. Let's try this again at the same NN model and see if there is any difference.

In [ ]:
KNN_NMF_TRAIN_SIZE = 200000
KNN_NMF_TEST_SIZE = 200

from sklearn.neighbors import KNeighborsClassifier
temp = []
for i in [1, 5]:
    knn_nmf = KNeighborsClassifier(n_neighbors=i, n_jobs=8)
    knn_nmf.fit(X_train_nmf[:KNN_NMF_TRAIN_SIZE], y_train[:KNN_NMF_TRAIN_SIZE])
    train_score_nmf = knn_nmf.score(X_train_nmf[:KNN_NMF_TEST_SIZE], y_train[:KNN_NMF_TEST_SIZE])
    test_score_nmf = knn_nmf.score(X_test_nmf[:KNN_NMF_TEST_SIZE], y_test[:KNN_NMF_TEST_SIZE])
    li = [i,train_score_nmf,test_score_nmf]
    temp.append(li)
temp

We got 59% accuracy now. A fair improvement over PCA.